In [1]:
from importlib import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./dracula.mplstyle')
import logging
logging.basicConfig()
LOG = logging.getLogger(__name__)
LOG.setLevel(logging.INFO)
LOG.info('Hello from notebook')

from pydataset import data
df = data('diamonds').reset_index().set_index('index')

import frankenfit as ff
reload(ff)

logging.getLogger('frankenfit').setLevel(logging.INFO)

INFO:__main__:Hello from notebook


In [38]:
client.shutdown()

In [22]:
from dask import distributed
cluster = distributed.LocalCluster(n_workers=4, threads_per_worker=2, host="devbox.homenet.thebanes.org")
print(cluster.dashboard_link)
client = distributed.Client(cluster)
client

#cluster = distributed.LocalCluster(n_workers=1, threads_per_worker=1)
#client = distributed.Client(cluster)
#client = distributed.Client()
#client

http://10.0.2.82:8787/status


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.0.2.82:8787/status,
Dashboard: http://10.0.2.82:8787/status,Workers: 4
Total threads: 8,Total memory: 23.47 GiB
Status: running,Using processes: True
Comm: tcp://10.0.2.82:38079,Workers: 4
Dashboard: http://10.0.2.82:8787/status,Total threads: 8
Started: Just now,Total memory: 23.47 GiB
Comm: tcp://10.0.2.82:44427,Total threads: 2
Dashboard: http://10.0.2.82:46467/status,Memory: 5.87 GiB
Nanny: tcp://10.0.2.82:45503,


In [23]:
backend = ff.DaskBackend(client)
backend

DaskBackend(address='tcp://10.0.2.82:38079')

2022-11-22 20:16:32,963 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [24]:
def foo(x):
    return f"foo({x})"

backend.submit("key_foo", foo, 42).result()

'foo(42)'

In [5]:
t = ff.ReadDataset('./diamonds.csv', format='csv', index_col='index')
t.fit(backend=backend)

FitTransform(resolved_transform=ReadDataset(tag='ReadDataset#1', paths=['./diamonds.csv'], columns=None, format='csv', filter=None, index_col='index', dataset_kwargs=None, scanner_kwargs=None), state=<class 'frankenfit.backend.DaskFuture'>, bindings={}, )

In [6]:
t.apply(backend=backend)

DaskFuture(dask_future=<Future: pending, key: ReadDataset#1._apply-0eef899b2f20547d7e4e78e2e064becd>)

In [7]:
t.apply(backend=backend).result()

,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56


In [13]:
t.fit(backend=backend).apply()


,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56


In [14]:
t.fit().apply(backend=backend).result()

,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56


In [ ]:
t.fit(backend=backend).apply(backend=backend).result()

In [9]:
t.fit(backend=backend).state().result()

In [15]:
(
    ff.DataFramePipeline()
    .read_dataset('./diamonds.csv', format='csv', index_col='index')
).fit(backend=backend).state().result()

[FitTransform(resolved_transform=ReadDataset(tag='ReadDataset#2', paths=['./diamonds.csv'], columns=None, format='csv', filter=None, index_col='index', dataset_kwargs=None, scanner_kwargs=None), state=<class 'NoneType'>, bindings={}, )]

In [17]:
t = ff.dataframe.ZScore(["price"])
#t.fit(df, backend=backend)  # warning about serializing df into task graph
t.fit(df).state()

{'means': price    3932.799722
 dtype: float64,
 'stddevs': price    3989.439738
 dtype: float64}

In [19]:
t = ff.dataframe.ZScore(["price"])
t.fit(df, backend=backend).state().result()  # warning about serializing df into task graph

{'means': price    3932.799722
 dtype: float64,
 'stddevs': price    3989.439738
 dtype: float64}

In [20]:
from frankenfit.dataframe import ZScore, Clip

In [21]:
# Future-passing
read = ff.ReadDataset('./diamonds.csv', format='csv', index_col='index')
zscore = ZScore(["price"])
clip = Clip(["price"], upper=2, lower=-2)

d = None
ft = read.fit(d, backend=backend)
d = ft.apply(d, backend=backend)

In [23]:
d.unwrap()

<Future: finished, type: pandas.core.frame.DataFrame, key: ReadDataset#3._apply-015a4e268e828d6fa06410dab9cd0e16>

In [24]:
ft = zscore.fit(d, backend=backend)
d = ft.apply(d, backend=backend)

In [25]:
d

DaskFuture(dask_future=<Future: finished, type: pandas.core.frame.DataFrame, key: ZScore#5._apply-3cd71d61bb74fb5988a22367638fc2cb>)

In [26]:
ft = clip.fit(d, backend=backend)
d = ft.apply(d, backend=backend)

In [27]:
ft.state()

DaskFuture(dask_future=<Future: finished, type: NoneType, key: Clip#1._fit-aaa21023699e8bcad91a45d484911827>)

In [28]:
d.result()

,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,-0.904087,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,-0.904087,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,-0.903836,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,-0.902081,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,-0.901831,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,-0.294728,5.75,5.76,3.50
53937,0.72,Good,D,SI1,63.1,55.0,-0.294728,5.69,5.75,3.61
53938,0.70,Very Good,D,SI1,62.8,60.0,-0.294728,5.66,5.68,3.56


In [25]:
t = (
    ff.ReadDataset('./diamonds.csv', format='csv', index_col='index')
    .then()  # something wrong with return type of then() here; type params not bound?
    .z_score(["price"])
    .clip(["price"], lower=-2, upper=2)
)
t

DataFramePipeline(tag='DataFramePipeline#11', transforms=[ReadDataset(tag='ReadDataset#2', paths=['./diamonds.csv'], columns=None, format='csv', filter=None, index_col='index', dataset_kwargs=None, scanner_kwargs=None), ZScore(tag='ZScore#3', cols=['price'], w_col=None), Clip(tag='Clip#3', cols=['price'], upper=2, lower=-2)])

In [26]:
tf = t.fit(backend=backend)
tf.state()

DaskFuture(dask_future=<Future: pending, key: DataFramePipeline#11._fit-7c37dbc8274fcd970206862c36d7504f>)

In [27]:
FEATURES = ["carat", "x", "y", "z", "depth", "table"]

def bake_features(cols):
    return (
        ff.DataFramePipeline()
        .print(fit_msg=f"Baking: {cols}")
        .winsorize(cols, limit=0.05)
        .z_score(cols)
        .impute_constant(cols, 0.0)
        .clip(cols, upper=2, lower=-2)
    )

ff.ReadDataset('./diamonds.csv', format='csv', index_col='index').then(bake_features(FEATURES)).apply().head()

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,-1.131845,Ideal,E,SI2,-0.207349,-1.206255,326,-1.345194,-1.345905,-1.343759
2,-1.131845,Premium,E,SI1,-1.663782,1.835534,326,-1.345194,-1.345905,-1.343759
3,-1.131845,Good,E,VS1,-2.000000,1.835534,327,-1.345194,-1.345905,-1.343759
4,-1.131845,Premium,I,VS2,0.563704,0.314640,334,-1.345194,-1.345905,-1.343759
5,-1.108324,Good,J,SI2,1.334757,0.314640,335,-1.297969,-1.298397,-1.190838


In [29]:
ff.ReadDataset('./diamonds.csv', format='csv', index_col='index').then(bake_features(FEATURES)).apply(backend=backend).result().head()

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,-1.131845,Ideal,E,SI2,-0.207349,-1.206255,326,-1.345194,-1.345905,-1.343759
2,-1.131845,Premium,E,SI1,-1.663782,1.835534,326,-1.345194,-1.345905,-1.343759
3,-1.131845,Good,E,VS1,-2.000000,1.835534,327,-1.345194,-1.345905,-1.343759
4,-1.131845,Premium,I,VS2,0.563704,0.314640,334,-1.345194,-1.345905,-1.343759
5,-1.108324,Good,J,SI2,1.334757,0.314640,335,-1.297969,-1.298397,-1.190838


In [30]:
from sklearn.linear_model import LinearRegression

FEATURES = ["carat", "x", "y", "z", "depth", "table"]

def bake_features(cols):
    return (
        ff.DataFramePipeline()
        .print(fit_msg=f"Baking: {cols}")
        .winsorize(cols, limit=0.05)
        .z_score(cols)
        .impute_constant(cols, 0.0)
        .clip(cols, upper=2, lower=-2)
    )

complex_pipeline = (
    ff.DataFramePipeline()[FEATURES + ["{response_col}"]]
    .copy("{response_col}", "{response_col}_train")
    .winsorize("{response_col}_train", limit=0.05)
    .pipe(["carat", "{response_col}_train"], np.log1p)
    #.then(bake_features(FEATURES))
    .if_hyperparam_is_true("bake_features", bake_features(FEATURES))
    .sk_learn(
        LinearRegression,
        # x_cols=["carat", "depth", "table"],
        x_cols=ff.HP("predictors"),
        response_col="{response_col}_train",
        hat_col="{response_col}_hat",
        class_params={"fit_intercept": True},
    )
    # transform {response_col}_hat from log-dollars back to dollars
    .copy("{response_col}_hat", "{response_col}_hat_dollars")
    .pipe("{response_col}_hat_dollars", np.expm1)
)

complex_pipeline.hyperparams()

{'bake_features', 'predictors', 'response_col'}

In [5]:
bindings = {"response_col": "price", "bake_features": True, "predictors": FEATURES}

In [6]:
read_diamonds = ff.ReadDataset('./diamonds.csv', format='csv', index_col='index')

In [15]:
#ff.ReadDataset('./diamonds.csv', format='csv', index_col='index').then(complex_pipeline).fit(bindings=bindings)
read_diamonds.then(complex_pipeline).fit(bindings=bindings).apply().head()

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


,carat,x,y,z,depth,table,price,price_train,price_hat,price_hat_dollars
index,,,,,,,,,,
1,-1.251924,-1.345194,-1.345905,-1.343759,-0.207349,-1.206255,326,6.300786,6.480260,651.140243
2,-1.251924,-1.345194,-1.345905,-1.343759,-1.663782,1.835534,326,6.300786,6.419302,612.574861
3,-1.251924,-1.345194,-1.345905,-1.343759,-2.000000,1.835534,327,6.300786,6.416375,610.781453
4,-1.251924,-1.345194,-1.345905,-1.343759,0.563704,0.314640,334,6.300786,6.462834,639.874609
5,-1.218600,-1.297969,-1.298397,-1.190838,1.334757,0.314640,335,6.300786,6.549435,697.848961


In [16]:
fit = read_diamonds.then(complex_pipeline).fit(bindings=bindings, backend=backend)
fit

FitTransform(resolved_transform=DataFramePipeline(tag='DataFramePipeline#10', transforms=[ReadDataset(tag='ReadDataset#1', paths=['./diamonds.csv'], columns=None, format='csv', filter=None, index_col='index', dataset_kwargs=None, scanner_kwargs=None), Select(tag='Select#2', cols=['carat', 'x', 'y', 'z', 'depth', 'table', '{response_col}']), Copy(tag='Copy#3', cols=['{response_col}'], dest_cols=['{response_col}_train']), Winsorize(tag='Winsorize#3', cols=['{response_col}_train'], limit=0.05), Pipe(tag='Pipe#3', cols=['carat', '{response_col}_train'], apply_fun=<ufunc 'log1p'>), IfHyperparamIsTrue(tag='IfHyperparamIsTrue#2', name='bake_features', then_transform=DataFramePipeline(tag='DataFramePipeline#7', transforms=[Print(tag='Print#2', fit_msg="Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']", apply_msg=None, dest=None), Winsorize(tag='Winsorize#4', cols=['carat', 'x', 'y', 'z', 'depth', 'table'], limit=0.05), ZScore(tag='ZScore#2', cols=['carat', 'x', 'y', 'z', 'depth', 'table'], w

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


In [17]:
fit.state()

DaskFuture(dask_future=<Future: finished, type: list, key: DataFramePipeline#10._fit-2ba2aa9359350f8f7dec92220d0f6e92>)

In [18]:
fit.apply(backend=backend).result().head()

,carat,x,y,z,depth,table,price,price_train,price_hat,price_hat_dollars
index,,,,,,,,,,
1,-1.251924,-1.345194,-1.345905,-1.343759,-0.207349,-1.206255,326,6.300786,6.480260,651.140243
2,-1.251924,-1.345194,-1.345905,-1.343759,-1.663782,1.835534,326,6.300786,6.419302,612.574861
3,-1.251924,-1.345194,-1.345905,-1.343759,-2.000000,1.835534,327,6.300786,6.416375,610.781453
4,-1.251924,-1.345194,-1.345905,-1.343759,0.563704,0.314640,334,6.300786,6.462834,639.874609
5,-1.218600,-1.297969,-1.298397,-1.190838,1.334757,0.314640,335,6.300786,6.549435,697.848961


In [20]:
fit.apply().head()

,carat,x,y,z,depth,table,price,price_train,price_hat,price_hat_dollars
index,,,,,,,,,,
1,-1.251924,-1.345194,-1.345905,-1.343759,-0.207349,-1.206255,326,6.300786,6.480260,651.140243
2,-1.251924,-1.345194,-1.345905,-1.343759,-1.663782,1.835534,326,6.300786,6.419302,612.574861
3,-1.251924,-1.345194,-1.345905,-1.343759,-2.000000,1.835534,327,6.300786,6.416375,610.781453
4,-1.251924,-1.345194,-1.345905,-1.343759,0.563704,0.314640,334,6.300786,6.462834,639.874609
5,-1.218600,-1.297969,-1.298397,-1.190838,1.334757,0.314640,335,6.300786,6.549435,697.848961


In [31]:
read_diamonds.then(complex_pipeline).apply(bindings=bindings, backend=backend).result().head()

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


,carat,x,y,z,depth,table,price,price_train,price_hat,price_hat_dollars
index,,,,,,,,,,
1,-1.251924,-1.345194,-1.345905,-1.343759,-0.207349,-1.206255,326,6.300786,6.480260,651.140243
2,-1.251924,-1.345194,-1.345905,-1.343759,-1.663782,1.835534,326,6.300786,6.419302,612.574861
3,-1.251924,-1.345194,-1.345905,-1.343759,-2.000000,1.835534,327,6.300786,6.416375,610.781453
4,-1.251924,-1.345194,-1.345905,-1.343759,0.563704,0.314640,334,6.300786,6.462834,639.874609
5,-1.218600,-1.297969,-1.298397,-1.190838,1.334757,0.314640,335,6.300786,6.549435,697.848961


In [32]:
pip = ff.ReadDataset('./diamonds.csv', format='csv', index_col='index').then(complex_pipeline)

In [36]:
pip.fit(bindings=bindings, backend=backend).apply(backend=backend)

DaskFuture(dask_future=<Future: pending, key: DataFramePipeline#21._apply-46d1c6ff5061667a0b2e2d1ceb4e2f2b>)

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


In [37]:
pip.apply(bindings=bindings, backend=backend)

DaskFuture(dask_future=<Future: pending, key: Pipe#6._apply-8042ac5a32107ddc58cd62d8789a701f>)

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


In [ ]:
# Good TODOs:
# Optimize StatelessTransform.fit() not to submit/always use DummyBackend?
# BasePipeline.fit() and FitTransform[BasePipeline, ...] should chain backend submissions, like BasePipeline.apply()
# Backend.fit()/apply() methods. apply() overloaded for FitTransform, StatelessTransform, BasePipeline.
# ForBindings: combine_fun param, maybe generic in DataResult; DataFramePipeline expects specialized for DataFrames
# Start parallelizing appropriate transforms: ForBindings, Join, GroupByCols, GroupByBindings